In [1]:
import os
import json
from PIL import Image, ImageDraw
import re
import numpy as np
import cv2
import imageio
import matplotlib.pyplot as plt
import tifffile as tiff
from tqdm import tqdm
from skimage.exposure import equalize_adapthist
from scipy.stats import stats
import matplotlib.animation as animation
import pandas as pd
import csv
import shutil
from skimage.morphology import dilation, erosion
from skimage import measure

from scipy.ndimage import center_of_mass
from glob import glob
import random

In [ ]:
# import numpy as np
# import matplotlib.pyplot as plt
# import imageio
# from skimage import measure



def analyze_mask(frame, mask_path_template, size_threshold=100):


    def make_np(img):
        return np.array(img)


    def plot_labels(original, ax, title=None,
                    txt_args={'color':'red', 'ha':'center', 'va':'center', 
                              'fontsize':'x-large', 'fontweight':'bold'},
                    cmap='gist_earth'):
        ax.imshow(original, cmap=cmap)
        labels = np.unique(original)
        print("Labels in plot_labels function:", labels)
        ax.set_title(title)
        for label in labels:
            if label != 0:  # Skip background label
                inds = np.argwhere(original == label)
                loc = inds.mean(0)
                ax.text(loc[1], loc[0], str(label), **txt_args)

    
    # Load the mask image using imageio
    mask_path = mask_path_template.format(frame=frame)
    mask = imageio.imread(mask_path)

    # Print the labels directly from the mask
    unique_labels = np.unique(mask)
    print("Labels from the mask:", unique_labels)

    # Measure the properties of the labeled regions
    properties = measure.regionprops(mask)

    # Extract areas and labels, review small cell sizes
    labeled_areas = [(prop.label, prop.area) for prop in properties if prop.label != 0]
    labels, areas = zip(*labeled_areas)

    # Print details of very small cells
    print("Reviewing small cell sizes (less than 100 pixels):")
    for label, area in zip(labels, areas):
        if area < 100:
            print(f"Cell label: {label}, Cell size: {area} pixels")

    # Apply a size threshold to filter out small artifacts
    filtered_areas = [(prop.label, prop.area) for prop in properties if prop.label != 0 and prop.area >= size_threshold]
    filtered_labels, filtered_areas = zip(*filtered_areas)

    # Calculate the area of the background
    background_area = np.sum(mask == 0)

    # Compute the average, min, and max cell sizes for filtered cells
    average_cell_size_filtered = np.mean(filtered_areas)
    min_cell_size_filtered = np.min(filtered_areas)
    max_cell_size_filtered = np.max(filtered_areas)
    min_cell_label_filtered = filtered_labels[np.argmin(filtered_areas)]
    max_cell_label_filtered = filtered_labels[np.argmax(filtered_areas)]

    # Display filtered results and areas of individual cells
    print(f"\nNumber of cells after applying size threshold: {len(filtered_areas)}")
    print(f"Average cell size: {average_cell_size_filtered} pixels")
    print(f"Minimum cell size: {min_cell_size_filtered} pixels (Label: {min_cell_label_filtered})")
    print(f"Maximum cell size: {max_cell_size_filtered} pixels (Label: {max_cell_label_filtered})")
    print(f"Background size: {background_area} pixels(Label: 0)")
    # print(f"Background label: 0")

    # Print areas of individual cells
    print("\nIndividual cell areas after filtering:")
    for label, area in zip(filtered_labels, filtered_areas):
        print(f"Cell label: {label}, Cell area: {area} pixels")

    # Plot the labels for filtered cells, excluding the background label
    fig, ax = plt.subplots(figsize=(10, 10))
    plot_labels(mask, ax, title="Labeled Mask with Correct Labels")
    plt.show()

# Example usage
frame = 410
mask_path_template = "/home/MinaHossain/Label_5004/6row_images_min_A/01_ST/SEG/man_seg{frame:04d}.tif"
# mask_path_template = "/home/MinaHossain/Label_5004/6row_images_min_A/02_GT/SEG/man_seg{frame:04d}.tif"
analyze_mask(frame, mask_path_template)





In [ ]:

# === PARAMETERS ===
# image_dir = "/home/MinaHossain/EmbedTrack/Shape_measure/HP4_TRA"  # <<< CHANGE 

# import os
# import numpy as np
# import tifffile
# import cv2
# from scipy.ndimage import center_of_mass
# from glob import glob
# import random

# === PARAMETERS ===
image_dir = "/home/MinaHossain/EmbedTrack/Shape_measure/HP4_TRA"  # <<< CHANGE THIS
output_video = "cell_4_tracking_colored_final.avi"
target_cell_id = 4
video_fps = 1
upscale_factor = 1.5

# === STEP 1: Build cell_locations + cell color map ===
cell_locations = {}
all_cell_ids = set()

# First pass: collect all cell IDs
mask_paths = sorted(glob(os.path.join(image_dir, "*.tif")))
for path in mask_paths:
    image_name = os.path.basename(path)
    mask = tifffile.imread(path)
    cell_ids = np.unique(mask)
    cell_ids = cell_ids[cell_ids != 0]
    all_cell_ids.update(cell_ids)

# Assign unique colors to each cell ID
cell_colors = {}
for cid in all_cell_ids:
    if cid == target_cell_id:
        cell_colors[cid] = (0, 0, 255)  # Bright RED for target cell
    else:
        cell_colors[cid] = tuple(random.randint(30, 220) for _ in range(3))  # Random distinct color

# Second pass: build centroid locations
for path in mask_paths:
    image_name = os.path.basename(path)
    mask = tifffile.imread(path)
    cell_ids = np.unique(mask)
    cell_ids = cell_ids[cell_ids != 0]

    cell_locations[image_name] = {}
    for cid in cell_ids:
        binary_mask = (mask == cid)
        if np.any(binary_mask):
            y, x = center_of_mass(binary_mask)
            cell_locations[image_name][cid] = (x, y)

# === STEP 2: Create video frames ===
frames = []
for frame_idx, image_name in enumerate(sorted(cell_locations.keys()), start=1):
    path = os.path.join(image_dir, image_name)
    mask = tifffile.imread(path)

    # Start with a black background
    color_mask = np.zeros((*mask.shape, 3), dtype=np.uint8)

    # Overlay each cell region with its assigned color
    for cid in np.unique(mask):
        if cid == 0:
            continue
        region = (mask == cid)
        color = cell_colors[cid]
        color_mask[region] = color

    # Add centroid markers and labels
    for cid, (x, y) in cell_locations[image_name].items():
        if cid == target_cell_id:
            # Large label for target cell
            cv2.circle(color_mask, (int(x), int(y)), 10, (255, 255, 255), -1)
            cv2.putText(color_mask, f"{cid}", (int(x)+10, int(y)-10),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.4, cell_colors[cid], 3)
        else:
            # Regular label for other cells
            cv2.putText(color_mask, f"{cid}", (int(x)+10, int(y)-10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, cell_colors[cid], 2)

    # Add frame and file name info
    cv2.putText(color_mask, f"Frame {frame_idx:03d}", (20, 40),
                cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255, 255, 255), 3)
    cv2.putText(color_mask, image_name, (20, 80),
                cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255, 255, 255), 2)

    # Upscale
    new_w = int(color_mask.shape[1] * upscale_factor)
    new_h = int(color_mask.shape[0] * upscale_factor)
    color_mask = cv2.resize(color_mask, (new_w, new_h), interpolation=cv2.INTER_NEAREST)

    frames.append(color_mask)

# === STEP 3: Save final video ===
if frames:
    height, width, _ = frames[0].shape
    print(f"Final video frame size: {width}x{height}")
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(output_video, fourcc, video_fps, (width, height))
    for frame in frames:
        out.write(frame)
    out.release()
    print(f"✅ Final color-enhanced annotated video saved: {output_video}")
else:
    print("❌ No frames found.")


Final video frame size: 3072x6934
✅ Final color-enhanced annotated video saved: cell_4_tracking_colored_final.avi


In [5]:
# import os
# import numpy as np
# import tifffile
# import cv2
# from scipy.ndimage import center_of_mass
# from glob import glob
# import random

# === PARAMETERS ===
# image_dir = "path/to/mask_images"  # <<< CHANGE THIS
image_dir = "/home/MinaHossain/EmbedTrack/Shape_measure/HP4_TRA"
output_dir = "/home/MinaHossain/EmbedTrack/"  # <<< OUTPUT FOLDER
target_cell_id = 4
output_video = os.path.join(output_dir, f"cell_{target_cell_id}_zoomed_tracking.avi")
output_frame_list_file = os.path.join(output_dir, f"cell_{target_cell_id}_frame_list.txt")
video_fps = 1
zoom_window_size = 256
upscale_factor = 2

# === Ensure output directory exists ===
os.makedirs(output_dir, exist_ok=True)

# === STEP 1: Build cell_locations and color mapping ===
cell_locations = {}
target_frames = []
all_cell_ids = set()

mask_paths = sorted(glob(os.path.join(image_dir, "*.tif")))

# Collect all cell IDs
for path in mask_paths:
    image_name = os.path.basename(path)
    mask = tifffile.imread(path)
    cell_ids = np.unique(mask)
    cell_ids = cell_ids[cell_ids != 0]
    all_cell_ids.update(cell_ids)

# Assign unique colors
cell_colors = {}
for cid in all_cell_ids:
    if cid == target_cell_id:
        cell_colors[cid] = (0, 0, 255)  # Bright red
    else:
        cell_colors[cid] = tuple(random.randint(50, 220) for _ in range(3))

# Get centroid positions
for path in mask_paths:
    image_name = os.path.basename(path)
    mask = tifffile.imread(path)
    cell_ids = np.unique(mask)
    cell_ids = cell_ids[cell_ids != 0]

    cell_locations[image_name] = {}
    for cid in cell_ids:
        binary_mask = (mask == cid)
        if np.any(binary_mask):
            y, x = center_of_mass(binary_mask)
            cell_locations[image_name][cid] = (x, y)

# === STEP 2: Create Zoomed-in Frames ===
frames = []

for frame_idx, image_name in enumerate(sorted(cell_locations.keys()), start=1):
    if target_cell_id not in cell_locations[image_name]:
        continue

    target_frames.append((frame_idx, image_name))
    path = os.path.join(image_dir, image_name)
    mask = tifffile.imread(path)

    h, w = mask.shape
    cx, cy = cell_locations[image_name][target_cell_id]

    # Crop window
    half = zoom_window_size // 2
    x1 = int(max(cx - half, 0))
    y1 = int(max(cy - half, 0))
    x2 = int(min(cx + half, w))
    y2 = int(min(cy + half, h))

    cropped_mask = mask[y1:y2, x1:x2]
    color_crop = np.zeros((*cropped_mask.shape, 3), dtype=np.uint8)

    # Color each cell in crop
    for cid in np.unique(cropped_mask):
        if cid == 0:
            continue
        region = (cropped_mask == cid)
        color_crop[region] = cell_colors[cid]

    # Annotate labels
    for cid, (x, y) in cell_locations[image_name].items():
        if x < x1 or x > x2 or y < y1 or y > y2:
            continue
        rel_x, rel_y = int(x - x1), int(y - y1)
        if cid == target_cell_id:
            cv2.circle(color_crop, (rel_x, rel_y), 10, (255, 255, 255), -1)
            cv2.putText(color_crop, f"{cid}", (rel_x + 10, rel_y - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.4, cell_colors[cid], 3)
        else:
            cv2.putText(color_crop, f"{cid}", (rel_x + 10, rel_y - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, cell_colors[cid], 2)

    # Frame info
    cv2.putText(color_crop, f"Frame {frame_idx:03d}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255, 255, 255), 2)
    cv2.putText(color_crop, image_name, (10, 60),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

    # Target cell presence label (highlighted)
    cv2.putText(color_crop, f"Target Cell ID {target_cell_id} Visible",
                (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 255, 255), 3)

    # Upscale
    color_crop = cv2.resize(color_crop, (int(color_crop.shape[1]*upscale_factor),
                                         int(color_crop.shape[0]*upscale_factor)), interpolation=cv2.INTER_NEAREST)
    frames.append(color_crop)

# === STEP 3: Write Video ===
if frames:
    height, width, _ = frames[0].shape
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(output_video, fourcc, video_fps, (width, height))
    for frame in frames:
        out.write(frame)
    out.release()
    print(f"✅ Zoomed-in annotated video saved: {output_video}")
else:
    print("❌ No frames found with target cell.")

# === STEP 4: Save Frame List ===
with open(output_frame_list_file, "w") as f:
    for frame_idx, image_name in target_frames:
        f.write(f"Frame {frame_idx:03d} - {image_name}\n")
print(f"✅ Frame list saved: {output_frame_list_file}")


✅ Zoomed-in annotated video saved: /home/MinaHossain/EmbedTrack/cell_4_zoomed_tracking.avi
✅ Frame list saved: /home/MinaHossain/EmbedTrack/cell_4_frame_list.txt


In [ ]:
# import os
# import numpy as np
# import tifffile
# import cv2
# import imageio
# import matplotlib.pyplot as plt
# from skimage import measure
# from scipy.ndimage import center_of_mass
# from glob import glob
# import random

# # === PARAMETERS ===
# image_dir = "/home/MinaHossain/EmbedTrack/Shape_measure/HP4_TRA"  # <<< CHANGE THIS
# output_dir = "/home/MinaHossain/EmbedTrack/Cell_Frame-Video"  # <<< OUTPUT FOLDER
# target_cell_id = 4
# output_video = os.path.join(output_dir, f"cell_{target_cell_id}_only_video.avi")
# output_frame_list_file = os.path.join(output_dir, f"cell_{target_cell_id}_accurate_frame_list.txt")
# video_fps = 1
# upscale_factor = 2

# # === Ensure output directory exists ===
# os.makedirs(output_dir, exist_ok=True)

# # === ANALYZE MASK FUNCTION (Simplified and Clean) ===
# def analyze_mask_from_path(mask_path, frame):
#     def plot_labels(original, ax, title=None,
#                     txt_args={'color': 'red', 'ha': 'center', 'va': 'center',
#                               'fontsize': 'x-large', 'fontweight': 'bold'},
#                     cmap='gist_earth'):
#         ax.imshow(original, cmap=cmap)
#         labels = np.unique(original)
#         print("Labels in plot_labels function:", labels)
#         ax.set_title(title)
#         for label in labels:
#             if label != 0:
#                 inds = np.argwhere(original == label)
#                 loc = inds.mean(0)
#                 ax.text(loc[1], loc[0], str(label), **txt_args)

#     try:
#         mask = imageio.imread(mask_path)
#         unique_labels = np.unique(mask)
#         print(f"Frame {frame} - Labels from the mask: {unique_labels}")
#         fig, ax = plt.subplots(figsize=(10, 10))
#         plot_labels(mask, ax, title=f"Labeled Mask (Frame {frame})")
#         # plt.show()
#     except Exception as e:
#         print(f"❌ Failed to analyze mask for frame {frame}: {e}")

# # === MAIN PROCESSING ===
# mask_paths = sorted(glob(os.path.join(image_dir, "*.tif")))
# cell_locations = {}
# target_frames = []
# all_cell_ids = set()

# # Collect all cell IDs and their centroid locations
# for path in mask_paths:
#     image_name = os.path.basename(path)
#     mask = tifffile.imread(path)

#     print(f"Checking {image_name} - Labels: {np.unique(mask)}")  # Diagnostic Print

#     cell_ids = np.unique(mask)
#     cell_ids = cell_ids[cell_ids != 0]
#     all_cell_ids.update(cell_ids)

#     cell_locations[image_name] = {}
#     for cid in cell_ids:
#         binary_mask = (mask == cid)
#         if np.any(binary_mask):
#             y, x = center_of_mass(binary_mask)
#             cell_locations[image_name][cid] = (x, y)

# # Assign distinct color to each cell
# cell_colors = {cid: (0, 0, 255) if cid == target_cell_id else tuple(random.randint(50, 230) for _ in range(3)) for cid in all_cell_ids}

# frames = []

# # Process only frames where target cell is present using cell_locations
# for frame_idx, path in enumerate(mask_paths, start=1):
#     image_name = os.path.basename(path)
#     mask = tifffile.imread(path)

#     # Use robust check
#     if target_cell_id not in cell_locations[image_name]:
#         print(f"Target cell ID {target_cell_id} NOT FOUND in {image_name}")  # Diagnostic
#         continue

#     print(f"✅ Target cell ID {target_cell_id} FOUND in {image_name}")  # Diagnostic
#     target_frames.append((frame_idx, image_name))
#     h, w = mask.shape
#     color_frame = np.zeros((h, w, 3), dtype=np.uint8)

#     for cid in np.unique(mask):
#         if cid == 0:
#             continue
#         region = (mask == cid)
#         color_frame[region] = cell_colors[cid]

#     for cid, (x, y) in cell_locations.get(image_name, {}).items():
#         label_size = 1.6 if cid == target_cell_id else 0.9
#         label_thickness = 3 if cid == target_cell_id else 2
#         label_color = cell_colors[cid]

#         if cid == target_cell_id:
#             cv2.circle(color_frame, (int(x), int(y)), 10, (255, 255, 255), -1)
#         cv2.putText(color_frame, f"{cid}", (int(x)+10, int(y)-10),
#                     cv2.FONT_HERSHEY_SIMPLEX, label_size, label_color, label_thickness)

#     cv2.putText(color_frame, f"Frame {frame_idx:03d}", (20, 50),
#                 cv2.FONT_HERSHEY_SIMPLEX, 1.8, (255, 255, 255), 4)
#     cv2.putText(color_frame, image_name, (20, 100),
#                 cv2.FONT_HERSHEY_SIMPLEX, 1.4, (255, 255, 255), 3)
#     cv2.putText(color_frame, f"Target Cell ID {target_cell_id} Visible", (20, 150),
#                 cv2.FONT_HERSHEY_SIMPLEX, 1.4, (0, 255, 255), 3)

#     # Run analysis/plotting function
#     analyze_mask_from_path(mask_path=path, frame=frame_idx)

#     # Upscale
#     color_frame = cv2.resize(color_frame,
#                              (int(w * upscale_factor), int(h * upscale_factor)),
#                              interpolation=cv2.INTER_NEAREST)
#     frames.append(color_frame)

# # Save video
# if frames:
#     height, width, _ = frames[0].shape
#     fourcc = cv2.VideoWriter_fourcc(*'XVID')
#     out = cv2.VideoWriter(output_video, fourcc, video_fps, (width, height))
#     for frame in frames:
#         out.write(frame)
#     out.release()
#     print(f"✅ Final video saved: {output_video}")
# else:
#     print("❌ No frames found for target cell.")

# # Save frame list
# with open(output_frame_list_file, "w") as f:
#     for frame_idx, image_name in target_frames:
#         f.write(f"Frame {frame_idx:03d} - {image_name}\n")

# print(f"✅ Accurate frame list saved: {output_frame_list_file}")
# print(f"✅ Total frames with target cell: {len(target_frames)}")


# Modify your tracking code to also store original mask label alongside assigned tracking ID in CSV?
# OR update your video generation code to use the CSV directly?


In [ ]:
# import os
# import numpy as np
# import tifffile
# import cv2
# import imageio
# import matplotlib.pyplot as plt
# from scipy.ndimage import center_of_mass
# from glob import glob
# import random

# # === PARAMETERS ===
# image_dir = "/home/MinaHossain/EmbedTrack/Shape_measure/HP4_TRA"  # <<< CHANGE THIS IF NEEDED
# output_dir = "/home/MinaHossain/EmbedTrack/Cell_Frame-Video"
# target_cell_id = 4
# output_video = os.path.join(output_dir, f"cell_{target_cell_id}_only_video.avi")
# output_frame_list_file = os.path.join(output_dir, f"cell_{target_cell_id}_accurate_frame_list.txt")
# video_fps = 1
# upscale_factor = 2

# # === Ensure output directory exists ===
# os.makedirs(output_dir, exist_ok=True)

# # === ANALYZE MASK FUNCTION (Simplified and Clean) ===
# def analyze_mask_from_path(mask_path, frame):
#     def plot_labels(original, ax, title=None,
#                     txt_args={'color': 'red', 'ha': 'center', 'va': 'center',
#                               'fontsize': 'x-large', 'fontweight': 'bold'},
#                     cmap='gist_earth'):
#         ax.imshow(original, cmap=cmap)
#         labels = np.unique(original)
#         print("Labels in plot_labels function:", labels)
#         ax.set_title(title)
#         for label in labels:
#             if label != 0:
#                 inds = np.argwhere(original == label)
#                 loc = inds.mean(0)
#                 ax.text(loc[1], loc[0], str(label), **txt_args)

#     try:
#         mask = imageio.imread(mask_path)
#         unique_labels = np.unique(mask)
#         print(f"Frame {frame} - Labels from the mask: {unique_labels}")
#         fig, ax = plt.subplots(figsize=(10, 10))
#         plot_labels(mask, ax, title=f"Labeled Mask (Frame {frame})")
#         plt.show()
#     except Exception as e:
#         print(f"❌ Failed to analyze mask for frame {frame}: {e}")

# # === MAIN PROCESSING ===
# mask_paths = sorted(glob(os.path.join(image_dir, "*.tif")))
# cell_locations = {}
# target_frames = []
# all_cell_ids = set()

# # Collect all cell IDs and their centroid locations
# for path in mask_paths:
#     image_name = os.path.basename(path)
#     mask = tifffile.imread(path)
#     cell_ids = np.unique(mask)
#     cell_ids = cell_ids[cell_ids != 0]
#     all_cell_ids.update(cell_ids)

#     cell_locations[image_name] = {}
#     for cid in cell_ids:
#         binary_mask = (mask == cid)
#         if np.any(binary_mask):
#             y, x = center_of_mass(binary_mask)
#             cell_locations[image_name][cid] = (x, y)

# # Assign distinct color to each cell
# cell_colors = {cid: (0, 0, 255) if cid == target_cell_id else tuple(random.randint(50, 230) for _ in range(3)) for cid in all_cell_ids}

# frames = []

# # Process only frames where target cell is present
# for frame_idx, path in enumerate(mask_paths, start=1):
#     image_name = os.path.basename(path)
#     mask = tifffile.imread(path)

#     if target_cell_id not in np.unique(mask):
#         continue

#     target_frames.append((frame_idx, image_name))
#     h, w = mask.shape
#     color_frame = np.zeros((h, w, 3), dtype=np.uint8)

#     for cid in np.unique(mask):
#         if cid == 0:
#             continue
#         region = (mask == cid)
#         color_frame[region] = cell_colors[cid]

#     for cid, (x, y) in cell_locations.get(image_name, {}).items():
#         label_size = 1.6 if cid == target_cell_id else 0.9
#         label_thickness = 3 if cid == target_cell_id else 2
#         label_color = cell_colors[cid]

#         if cid == target_cell_id:
#             cv2.circle(color_frame, (int(x), int(y)), 10, (255, 255, 255), -1)
#         cv2.putText(color_frame, f"{cid}", (int(x)+10, int(y)-10),
#                     cv2.FONT_HERSHEY_SIMPLEX, label_size, label_color, label_thickness)

#     cv2.putText(color_frame, f"Frame {frame_idx:03d}", (20, 50),
#                 cv2.FONT_HERSHEY_SIMPLEX, 1.8, (255, 255, 255), 4)
#     cv2.putText(color_frame, image_name, (20, 100),
#                 cv2.FONT_HERSHEY_SIMPLEX, 1.4, (255, 255, 255), 3)
#     cv2.putText(color_frame, f"Target Cell ID {target_cell_id} Visible", (20, 150),
#                 cv2.FONT_HERSHEY_SIMPLEX, 1.4, (0, 255, 255), 3)

#     # Run analysis/plotting function
#     analyze_mask_from_path(mask_path=path, frame=frame_idx)

#     # Upscale
#     color_frame = cv2.resize(color_frame,
#                              (int(w * upscale_factor), int(h * upscale_factor)),
#                              interpolation=cv2.INTER_NEAREST)
#     frames.append(color_frame)

# # Save video
# if frames:
#     height, width, _ = frames[0].shape
#     fourcc = cv2.VideoWriter_fourcc(*'XVID')
#     out = cv2.VideoWriter(output_video, fourcc, video_fps, (width, height))
#     for frame in frames:
#         out.write(frame)
#     out.release()
#     print(f"✅ Final video saved: {output_video}")
# else:
#     print("❌ No frames found for target cell.")

# # Save frame list
# with open(output_frame_list_file, "w") as f:
#     for frame_idx, image_name in target_frames:
#         f.write(f"Frame {frame_idx:03d} - {image_name}\n")

# print(f"✅ Accurate frame list saved: {output_frame_list_file}")
# print(f"✅ Total frames with target cell: {len(target_frames)}")

# Modify your tracking code to also store original mask label alongside assigned tracking ID in CSV?
# OR update your video generation code to use the CSV directly?

In [14]:
import os
import cv2
import pandas as pd
import numpy as np
from glob import glob


# === PARAMETERS ===
csv_path = "/home/MinaHossain/EmbedTrack/PCA_t-SNE_PHATE/Cells_Centroid_Velocity_MA_5.csv"  # <<< Adjust if needed
image_dir = "/home/MinaHossain/EmbedTrack/Shape_measure/HP4_TRA"  # Directory containing raw mask images
output_dir = "/home/MinaHossain/EmbedTrack/Cell_Frame-Video"
target_cell_id = 4
output_video = os.path.join(output_dir, f"Cell_{target_cell_id}_CSV_based_tracking.avi")
output_frame_list = os.path.join(output_dir, f"Cell_{target_cell_id}_CSV_frame_list.txt")
video_fps = 1
upscale_factor = 2

# === Ensure output directory exists ===
os.makedirs(output_dir, exist_ok=True)

# === Load CSV data ===
df = pd.read_csv(csv_path)

# Extract data for all cells & target cell
df_all = df.copy()
df_target = df_all[df_all['Cell Number'] == target_cell_id].copy()

# Get list of frames where target cell appears
target_frames = sorted(df_target['Frame'].unique())
trajectory_points = []

# Prepare video frames
frames = []

# Map all frame image files
mask_paths = sorted(glob(os.path.join(image_dir, "*.tif")))
mask_map = {}
for path in mask_paths:
    frame_num = int(''.join(filter(str.isdigit, os.path.basename(path))))
    mask_map[frame_num] = path

for frame_num in target_frames:
    if frame_num not in mask_map:
        print(f"❌ Frame {frame_num} not found in masks, skipping...")
        continue

    # Read image
    mask = cv2.imread(mask_map[frame_num], cv2.IMREAD_GRAYSCALE)
    h, w = mask.shape
    frame_bgr = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)

    # Get all cells in this frame from CSV
    df_frame = df_all[df_all['Frame'] == frame_num]

    for _, row in df_frame.iterrows():
        cid = int(row['Cell Number'])
        cx, cy = int(row['Centroid_X_MA']), int(row['Centroid_Y_MA'])

        if cid == target_cell_id:
            # Highlight target cell
            cv2.circle(frame_bgr, (cx, cy), 10, (255, 255, 255), -1)
            cv2.putText(frame_bgr, f"{cid}", (cx+10, cy-10),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.6, (0, 0, 255), 3)
            trajectory_points.append((cx, cy))
        else:
            cv2.putText(frame_bgr, f"{cid}", (cx+5, cy-5),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    # Draw trajectory of target cell
    for i in range(1, len(trajectory_points)):
        cv2.line(frame_bgr, trajectory_points[i-1], trajectory_points[i], (0, 0, 255), 2)

    # Add frame label
    cv2.putText(frame_bgr, f"Frame {frame_num:03d}", (20, 40),
                cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255, 255, 255), 3)

    # Upscale
    frame_bgr = cv2.resize(frame_bgr, (int(w*upscale_factor), int(h*upscale_factor)), interpolation=cv2.INTER_NEAREST)
    frames.append(frame_bgr)

# === Save video ===
if frames:
    height, width, _ = frames[0].shape
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(output_video, fourcc, video_fps, (width, height))
    for frame in frames:
        out.write(frame)
    out.release()
    print(f"✅ Video saved: {output_video}")
else:
    print("❌ No frames with target cell found, video not generated.")

# === Save target frame list ===
with open(output_frame_list, "w") as f:
    for frame_num in target_frames:
        f.write(f"{frame_num}\n")

print(f"✅ Target cell frame list saved: {output_frame_list}")
print(f"✅ Total frames with target cell: {len(target_frames)}")



✅ Video saved: /home/MinaHossain/EmbedTrack/Cell_Frame-Video/Cell_4_CSV_based_tracking.avi
✅ Target cell frame list saved: /home/MinaHossain/EmbedTrack/Cell_Frame-Video/Cell_4_CSV_frame_list.txt
✅ Total frames with target cell: 69


[mpeg4 @ 0x498b9800] dimensions too large for MPEG-4
[ERROR:0@5282.397] global cap_ffmpeg_impl.hpp:3194 open Could not open codec mpeg4, error: Unspecified error (-22)
[ERROR:0@5282.397] global cap_ffmpeg_impl.hpp:3211 open VIDEOIO/FFMPEG: Failed to initialize VideoWriter

(python:1444257): GStreamer-CRITICAL **: 06:46:29.218: gst_element_make_from_uri: assertion 'gst_uri_is_valid (uri)' failed
[ WARN:0@5282.397] global cap_gstreamer.cpp:2617 open OpenCV | GStreamer warning: cannot link elements


In [15]:
import os
import cv2
import pandas as pd
import numpy as np
from glob import glob


# === PARAMETERS ===
csv_path = "/home/MinaHossain/EmbedTrack/PCA_t-SNE_PHATE/Cells_Centroid_Velocity_MA_5.csv"  # <<< Adjust if needed
image_dir = "/home/MinaHossain/EmbedTrack/Shape_measure/HP4_TRA"  # Directory containing raw mask images
output_dir = "/home/MinaHossain/EmbedTrack/Cell_Frame-Video"
target_cell_id = 4
output_video = os.path.join(output_dir, f"Cell_{target_cell_id}_CSV_based_tracking.avi")
output_frame_list = os.path.join(output_dir, f"Cell_{target_cell_id}_CSV_frame_list.txt")
video_fps = 1
upscale_factor = 2

# === Ensure output directory exists ===
os.makedirs(output_dir, exist_ok=True)

# === Load CSV data ===
df = pd.read_csv(csv_path)

# Extract data for all cells & target cell
df_all = df.copy()
df_target = df_all[df_all['Cell Number'] == target_cell_id].copy()

# Get list of frames where target cell appears
target_frames = sorted(df_target['Frame'].unique())
trajectory_points = []

# Prepare video frames
frames = []

# Map all frame image files
mask_paths = sorted(glob(os.path.join(image_dir, "*.tif")))
mask_map = {}
for path in mask_paths:
    try:
        fname = os.path.basename(path)
        digits = ''.join(filter(str.isdigit, fname))
        frame_num = int(digits)
        mask_map[frame_num] = path
    except:
        print(f"⚠️ Failed to parse frame number from: {path}")

for frame_num in target_frames:
    if frame_num not in mask_map:
        print(f"❌ Frame {frame_num} not found in masks, skipping...")
        continue

    print(f"✅ Drawing Cell {target_cell_id} in Frame {frame_num}")
    mask_path = mask_map[frame_num]

    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    if mask is None:
        print(f"⚠️ Could not read mask for frame {frame_num}")
        continue

    h, w = mask.shape
    frame_bgr = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)

    df_frame = df_all[df_all['Frame'] == frame_num]

    for _, row in df_frame.iterrows():
        cid = int(row['Cell Number'])
        cx, cy = int(row['Centroid_X_MA']), int(row['Centroid_Y_MA'])

        if cx >= w or cy >= h or cx < 0 or cy < 0:
            print(f"⚠️ Skipping invalid centroid {cx},{cy} for Cell {cid} in Frame {frame_num}")
            continue

        if cid == target_cell_id:
            cv2.circle(frame_bgr, (cx, cy), 10, (255, 255, 255), -1)
            cv2.putText(frame_bgr, f"{cid}", (cx+10, cy-10),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.6, (0, 0, 255), 3)
            trajectory_points.append((cx, cy))
        else:
            cv2.putText(frame_bgr, f"{cid}", (cx+5, cy-5),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    for i in range(1, len(trajectory_points)):
        cv2.line(frame_bgr, trajectory_points[i-1], trajectory_points[i], (0, 0, 255), 2)

    cv2.putText(frame_bgr, f"Frame {frame_num:03d}", (20, 40),
                cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255, 255, 255), 3)

    frame_bgr = cv2.resize(frame_bgr, (int(w*upscale_factor), int(h*upscale_factor)), interpolation=cv2.INTER_NEAREST)
    frames.append(frame_bgr)

# Save video
if frames:
    height, width, _ = frames[0].shape
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(output_video, fourcc, video_fps, (width, height))
    for frame in frames:
        out.write(frame)
    out.release()
    print(f"✅ Final video saved: {output_video}")
else:
    print("❌ No frames successfully drawn. Video not generated.")

# Save target frame list
with open(output_frame_list, "w") as f:
    for frame_num in target_frames:
        f.write(f"{frame_num}\n")

print(f"✅ Target cell frame list saved: {output_frame_list}")
print(f"✅ Total frames with target cell: {len(target_frames)}")



✅ Drawing Cell 4 in Frame 2
✅ Drawing Cell 4 in Frame 3
✅ Drawing Cell 4 in Frame 4
✅ Drawing Cell 4 in Frame 6
✅ Drawing Cell 4 in Frame 7
✅ Drawing Cell 4 in Frame 8
✅ Drawing Cell 4 in Frame 10
✅ Drawing Cell 4 in Frame 11
✅ Drawing Cell 4 in Frame 19
✅ Drawing Cell 4 in Frame 20
✅ Drawing Cell 4 in Frame 21
✅ Drawing Cell 4 in Frame 23
✅ Drawing Cell 4 in Frame 24
✅ Drawing Cell 4 in Frame 25
✅ Drawing Cell 4 in Frame 26
✅ Drawing Cell 4 in Frame 28
✅ Drawing Cell 4 in Frame 29
✅ Drawing Cell 4 in Frame 30
✅ Drawing Cell 4 in Frame 31
✅ Drawing Cell 4 in Frame 32
✅ Drawing Cell 4 in Frame 33
✅ Drawing Cell 4 in Frame 34
✅ Drawing Cell 4 in Frame 35
✅ Drawing Cell 4 in Frame 36
✅ Drawing Cell 4 in Frame 37
✅ Drawing Cell 4 in Frame 38
✅ Drawing Cell 4 in Frame 43
✅ Drawing Cell 4 in Frame 45
✅ Drawing Cell 4 in Frame 46
✅ Drawing Cell 4 in Frame 47
✅ Drawing Cell 4 in Frame 48
✅ Drawing Cell 4 in Frame 50
✅ Drawing Cell 4 in Frame 51
✅ Drawing Cell 4 in Frame 52
✅ Drawing Cell 4 in 

[mpeg4 @ 0x4ac87f00] dimensions too large for MPEG-4
[ERROR:0@5481.193] global cap_ffmpeg_impl.hpp:3194 open Could not open codec mpeg4, error: Unspecified error (-22)
[ERROR:0@5481.193] global cap_ffmpeg_impl.hpp:3211 open VIDEOIO/FFMPEG: Failed to initialize VideoWriter

(python:1444257): GStreamer-CRITICAL **: 06:49:48.014: gst_element_make_from_uri: assertion 'gst_uri_is_valid (uri)' failed
[ WARN:0@5481.193] global cap_gstreamer.cpp:2617 open OpenCV | GStreamer warning: cannot link elements


In [16]:
import os
import cv2
import pandas as pd
import numpy as np
from glob import glob


# === PARAMETERS ===
csv_path = "/home/MinaHossain/EmbedTrack/PCA_t-SNE_PHATE/Cells_Centroid_Velocity_MA_5.csv"  # <<< Adjust if needed
image_dir = "/home/MinaHossain/EmbedTrack/Shape_measure/HP4_TRA"  # Directory containing raw mask images
output_dir = "/home/MinaHossain/EmbedTrack/Cell_Frame-Video"
target_cell_id = 4
output_video = os.path.join(output_dir, f"Cell_{target_cell_id}_CSV_tracking_MJPG.avi")
output_frame_list = os.path.join(output_dir, f"Cell_{target_cell_id}_CSV_frame_list.txt")
video_fps = 1
upscale_factor = 1.5  # Reduced to avoid codec error

# === Ensure output directory exists ===
os.makedirs(output_dir, exist_ok=True)

# === Load CSV data ===
df = pd.read_csv(csv_path)
df_all = df.copy()
df_target = df_all[df_all['Cell Number'] == target_cell_id].copy()

target_frames = sorted(df_target['Frame'].unique())
trajectory_points = []
frames = []

mask_paths = sorted(glob(os.path.join(image_dir, "*.tif")))
mask_map = {}
for path in mask_paths:
    try:
        fname = os.path.basename(path)
        digits = ''.join(filter(str.isdigit, fname))
        frame_num = int(digits)
        mask_map[frame_num] = path
    except:
        print(f"⚠️ Failed to parse frame number from: {path}")

for frame_num in target_frames:
    if frame_num not in mask_map:
        print(f"❌ Frame {frame_num} not found in masks, skipping...")
        continue

    print(f"✅ Drawing Cell {target_cell_id} in Frame {frame_num}")
    mask_path = mask_map[frame_num]
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    if mask is None:
        print(f"⚠️ Could not read mask for frame {frame_num}")
        continue

    h, w = mask.shape
    frame_bgr = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)
    df_frame = df_all[df_all['Frame'] == frame_num]

    for _, row in df_frame.iterrows():
        cid = int(row['Cell Number'])
        cx, cy = int(row['Centroid_X_MA']), int(row['Centroid_Y_MA'])

        if cx >= w or cy >= h or cx < 0 or cy < 0:
            print(f"⚠️ Skipping invalid centroid {cx},{cy} for Cell {cid} in Frame {frame_num}")
            continue

        if cid == target_cell_id:
            cv2.circle(frame_bgr, (cx, cy), 10, (255, 255, 255), -1)
            cv2.putText(frame_bgr, f"{cid}", (cx+10, cy-10),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.6, (0, 0, 255), 3)
            trajectory_points.append((cx, cy))
        else:
            cv2.putText(frame_bgr, f"{cid}", (cx+5, cy-5),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    for i in range(1, len(trajectory_points)):
        cv2.line(frame_bgr, trajectory_points[i-1], trajectory_points[i], (0, 0, 255), 2)

    cv2.putText(frame_bgr, f"Frame {frame_num:03d}", (20, 40),
                cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255, 255, 255), 3)

    frame_bgr = cv2.resize(frame_bgr,
                           (int(w*upscale_factor), int(h*upscale_factor)),
                           interpolation=cv2.INTER_NEAREST)
    frames.append(frame_bgr)

# === Save video with MJPG codec ===
if frames:
    height, width, _ = frames[0].shape
    print(f"Video dimensions: {width}x{height}")
    fourcc = cv2.VideoWriter_fourcc(*'MJPG')
    out = cv2.VideoWriter(output_video, fourcc, video_fps, (width, height))
    for frame in frames:
        out.write(frame)
    out.release()
    print(f"✅ Final video saved using MJPG codec: {output_video}")
else:
    print("❌ No frames successfully drawn. Video not generated.")

# Save target frame list
with open(output_frame_list, "w") as f:
    for frame_num in target_frames:
        f.write(f"{frame_num}\n")

print(f"✅ Target cell frame list saved: {output_frame_list}")
print(f"✅ Total frames with target cell: {len(target_frames)}")


✅ Drawing Cell 4 in Frame 2
✅ Drawing Cell 4 in Frame 3
✅ Drawing Cell 4 in Frame 4
✅ Drawing Cell 4 in Frame 6
✅ Drawing Cell 4 in Frame 7
✅ Drawing Cell 4 in Frame 8
✅ Drawing Cell 4 in Frame 10
✅ Drawing Cell 4 in Frame 11
✅ Drawing Cell 4 in Frame 19
✅ Drawing Cell 4 in Frame 20
✅ Drawing Cell 4 in Frame 21
✅ Drawing Cell 4 in Frame 23
✅ Drawing Cell 4 in Frame 24
✅ Drawing Cell 4 in Frame 25
✅ Drawing Cell 4 in Frame 26
✅ Drawing Cell 4 in Frame 28
✅ Drawing Cell 4 in Frame 29
✅ Drawing Cell 4 in Frame 30
✅ Drawing Cell 4 in Frame 31
✅ Drawing Cell 4 in Frame 32
✅ Drawing Cell 4 in Frame 33
✅ Drawing Cell 4 in Frame 34
✅ Drawing Cell 4 in Frame 35
✅ Drawing Cell 4 in Frame 36
✅ Drawing Cell 4 in Frame 37
✅ Drawing Cell 4 in Frame 38
✅ Drawing Cell 4 in Frame 43
✅ Drawing Cell 4 in Frame 45
✅ Drawing Cell 4 in Frame 46
✅ Drawing Cell 4 in Frame 47
✅ Drawing Cell 4 in Frame 48
✅ Drawing Cell 4 in Frame 50
✅ Drawing Cell 4 in Frame 51
✅ Drawing Cell 4 in Frame 52
✅ Drawing Cell 4 in 

In [17]:
import os
import cv2
import pandas as pd
import numpy as np
from glob import glob

# === PARAMETERS ===
csv_path = "/home/MinaHossain/EmbedTrack/PCA_t-SNE_PHATE/Cells_Centroid_Velocity_MA_5.csv"  # <<< Adjust if needed
image_dir = "/home/MinaHossain/EmbedTrack/Shape_measure/HP4_TRA"  # Directory containing raw mask images
output_dir = "/home/MinaHossain/EmbedTrack/Cell_Frame-Video"

target_cell_id = 4
output_video = os.path.join(output_dir, f"Cell_{target_cell_id}_CSV_tracking_MJPG_Enhanced.avi")
output_frame_list = os.path.join(output_dir, f"Cell_{target_cell_id}_CSV_frame_list.txt")
video_fps = 1
upscale_factor = 1.5

# === Ensure output directory exists ===
os.makedirs(output_dir, exist_ok=True)

# === Load CSV data ===
df = pd.read_csv(csv_path)
df_all = df.copy()
df_target = df_all[df_all['Cell Number'] == target_cell_id].copy()
target_frames = sorted(df_target['Frame'].unique())
trajectory_points = []
frames = []

# === Map all frame image files ===
mask_paths = sorted(glob(os.path.join(image_dir, "*.tif")))
mask_map = {}
for path in mask_paths:
    try:
        fname = os.path.basename(path)
        digits = ''.join(filter(str.isdigit, fname))
        frame_num = int(digits)
        mask_map[frame_num] = path
    except:
        print(f"⚠️ Failed to parse frame number from: {path}")

# === Process frames ===
for frame_num in target_frames:
    if frame_num not in mask_map:
        print(f"❌ Frame {frame_num} not found in masks, skipping...")
        continue

    print(f"✅ Drawing Cell {target_cell_id} in Frame {frame_num}")
    mask_path = mask_map[frame_num]
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    if mask is None:
        print(f"⚠️ Could not read mask for frame {frame_num}")
        continue

    h, w = mask.shape
    frame_bgr = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)
    df_frame = df_all[df_all['Frame'] == frame_num]

    for _, row in df_frame.iterrows():
        cid = int(row['Cell Number'])
        cx, cy = int(row['Centroid_X_MA']), int(row['Centroid_Y_MA'])
        if cx >= w or cy >= h or cx < 0 or cy < 0:
            print(f"⚠️ Skipping invalid centroid {cx},{cy} for Cell {cid} in Frame {frame_num}")
            continue

        if cid == target_cell_id:
            # Simulate visual cell area
            radius = 25  # Or dynamic: int(np.sqrt(row["Area_MA"]) / 2)
            cv2.circle(frame_bgr, (cx, cy), radius, (0, 0, 255), 3)
            cv2.circle(frame_bgr, (cx, cy), 12, (255, 255, 255), -1)
            cv2.putText(frame_bgr, f"{cid}", (cx+12, cy-12),
                        cv2.FONT_HERSHEY_SIMPLEX, 2.0, (0, 0, 255), 5)
            trajectory_points.append((cx, cy))
        else:
            font_size = 1.0
            cv2.putText(frame_bgr, f"{cid}", (cx+5, cy-5),
                        cv2.FONT_HERSHEY_SIMPLEX, font_size, (0, 255, 0), 2)

    for i in range(1, len(trajectory_points)):
        cv2.line(frame_bgr, trajectory_points[i-1], trajectory_points[i], (0, 0, 255), 2)

    cv2.putText(frame_bgr, f"Frame {frame_num:03d}", (20, 40),
                cv2.FONT_HERSHEY_SIMPLEX, 1.6, (255, 255, 255), 4)

    frame_bgr = cv2.resize(frame_bgr,
                           (int(w*upscale_factor), int(h*upscale_factor)),
                           interpolation=cv2.INTER_NEAREST)
    frames.append(frame_bgr)

# === Save video with MJPG codec ===
if frames:
    height, width, _ = frames[0].shape
    print(f"Video dimensions: {width}x{height}")
    fourcc = cv2.VideoWriter_fourcc(*'MJPG')
    out = cv2.VideoWriter(output_video, fourcc, video_fps, (width, height))
    for frame in frames:
        out.write(frame)
    out.release()
    print(f"✅ Final enhanced video saved: {output_video}")
else:
    print("❌ No frames successfully drawn. Video not generated.")

# === Save frame list ===
with open(output_frame_list, "w") as f:
    for frame_num in target_frames:
        f.write(f"{frame_num}\n")

print(f"✅ Target cell frame list saved: {output_frame_list}")
print(f"✅ Total frames with target cell: {len(target_frames)}")


✅ Drawing Cell 4 in Frame 2
✅ Drawing Cell 4 in Frame 3
✅ Drawing Cell 4 in Frame 4
✅ Drawing Cell 4 in Frame 6
✅ Drawing Cell 4 in Frame 7
✅ Drawing Cell 4 in Frame 8
✅ Drawing Cell 4 in Frame 10
✅ Drawing Cell 4 in Frame 11
✅ Drawing Cell 4 in Frame 19
✅ Drawing Cell 4 in Frame 20
✅ Drawing Cell 4 in Frame 21
✅ Drawing Cell 4 in Frame 23
✅ Drawing Cell 4 in Frame 24
✅ Drawing Cell 4 in Frame 25
✅ Drawing Cell 4 in Frame 26
✅ Drawing Cell 4 in Frame 28
✅ Drawing Cell 4 in Frame 29
✅ Drawing Cell 4 in Frame 30
✅ Drawing Cell 4 in Frame 31
✅ Drawing Cell 4 in Frame 32
✅ Drawing Cell 4 in Frame 33
✅ Drawing Cell 4 in Frame 34
✅ Drawing Cell 4 in Frame 35
✅ Drawing Cell 4 in Frame 36
✅ Drawing Cell 4 in Frame 37
✅ Drawing Cell 4 in Frame 38
✅ Drawing Cell 4 in Frame 43
✅ Drawing Cell 4 in Frame 45
✅ Drawing Cell 4 in Frame 46
✅ Drawing Cell 4 in Frame 47
✅ Drawing Cell 4 in Frame 48
✅ Drawing Cell 4 in Frame 50
✅ Drawing Cell 4 in Frame 51
✅ Drawing Cell 4 in Frame 52
✅ Drawing Cell 4 in 

In [18]:
# def analyze_mask_from_path(mask_path, frame):
#     def plot_labels(original, ax, title=None,
#                     txt_args={'color': 'red', 'ha': 'center', 'va': 'center',
#                               'fontsize': 'x-large', 'fontweight': 'bold'},
#                     cmap='gist_earth'):
#         ax.imshow(original, cmap=cmap)
#         labels = np.unique(original)
#         print("Labels in plot_labels function:", labels)
#         ax.set_title(title)
#         for label in labels:
#             if label != 0:
#                 inds = np.argwhere(original == label)
#                 loc = inds.mean(0)
#                 ax.text(loc[1], loc[0], str(label), **txt_args)

#     try:
#         mask = imageio.imread(mask_path)
#         unique_labels = np.unique(mask)
#         print(f"Frame {frame} - Labels from the mask: {unique_labels}")
#         fig, ax = plt.subplots(figsize=(10, 10))
#         plot_labels(mask, ax, title=f"Labeled Mask (Frame {frame})")
#         # plt.show()
#     except Exception as e:
#         print(f"❌ Failed to analyze mask for frame {frame}: {e}")

In [19]:
import os
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
import imageio

# === PARAMETERS ===
# === PARAMETERS ===
csv_path = "/home/MinaHossain/EmbedTrack/PCA_t-SNE_PHATE/Cells_Centroid_Velocity_MA_5.csv"  # <<< Adjust if needed
image_dir = "/home/MinaHossain/EmbedTrack/Shape_measure/HP4_TRA"  # Directory containing raw mask images
output_dir = "/home/MinaHossain/EmbedTrack/Cell_Frame-Video"

temp_plot_dir = os.path.join(output_dir, "temp_plots")
target_cell_id = 4
output_video = os.path.join(output_dir, f"Cell_{target_cell_id}_Matplotlib_Labeled_Video.avi")
output_frame_list = os.path.join(output_dir, f"Cell_{target_cell_id}_CSV_frame_list.txt")
video_fps = 1

# === Ensure directories exist ===
os.makedirs(output_dir, exist_ok=True)
os.makedirs(temp_plot_dir, exist_ok=True)

# === Load CSV data ===
df = pd.read_csv(csv_path)
df_target = df[df["Cell Number"] == target_cell_id]
target_frames = sorted(df_target["Frame"].unique())

# === Create mapping of frame to mask file ===
mask_paths = sorted(glob(os.path.join(image_dir, "*.tif")))
mask_map = {}
for path in mask_paths:
    try:
        fname = os.path.basename(path)
        digits = ''.join(filter(str.isdigit, fname))
        frame_num = int(digits)
        mask_map[frame_num] = path
    except:
        print(f"⚠️ Could not parse frame number from {path}")

# === Define plot_labels function ===
def plot_labels(original, ax, title=None,
                txt_args={'color': 'red', 'ha': 'center', 'va': 'center',
                          'fontsize': 'xx-large', 'fontweight': 'bold'},
                cmap='gist_earth'):
    ax.imshow(original, cmap=cmap)
    labels = np.unique(original)
    print(f"Labels in plot_labels function for frame {title}: {labels}")
    ax.set_title(title)
    for label in labels:
        if label != 0:
            inds = np.argwhere(original == label)
            loc = inds.mean(0)
            ax.text(loc[1], loc[0], str(label), **txt_args)
    ax.axis("off")

# === Create frames using matplotlib ===
plot_frame_paths = []
for frame_num in target_frames:
    if frame_num not in mask_map:
        print(f"❌ Frame {frame_num} not found in mask map.")
        continue

    mask = imageio.imread(mask_map[frame_num])
    fig, ax = plt.subplots(figsize=(8, 8))
    plot_labels(mask, ax, title=f"Frame {frame_num} - Cell {target_cell_id}")
    plot_frame_path = os.path.join(temp_plot_dir, f"frame_{frame_num:03d}.png")
    fig.savefig(plot_frame_path, bbox_inches="tight")
    plt.close(fig)
    plot_frame_paths.append(plot_frame_path)

# === Read images and write video ===
if plot_frame_paths:
    sample_img = cv2.imread(plot_frame_paths[0])
    height, width, _ = sample_img.shape
    print(f"Video frame size: {width}x{height}")
    fourcc = cv2.VideoWriter_fourcc(*'MJPG')
    out = cv2.VideoWriter(output_video, fourcc, video_fps, (width, height))

    for frame_img_path in plot_frame_paths:
        img = cv2.imread(frame_img_path)
        out.write(img)
    out.release()
    print(f"✅ Final enhanced matplotlib-labeled video saved: {output_video}")
else:
    print("❌ No video frames generated. Video not saved.")

# === Save frame list ===
with open(output_frame_list, "w") as f:
    for frame_num in target_frames:
        f.write(f"{frame_num}\n")

print(f"✅ Frame list saved to: {output_frame_list}")
print(f"✅ Total frames with target cell: {len(target_frames)}")


/tmp/ipykernel_1444257/2430220196.py:65: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  mask = imageio.imread(mask_map[frame_num])


Labels in plot_labels function for frame Frame 2 - Cell 4: [ 0  2  3  4  5  6  7  8  9 10 11 12 14]
Labels in plot_labels function for frame Frame 3 - Cell 4: [ 0  2  3  4  5  6  7  8  9 10 11 12 14]
Labels in plot_labels function for frame Frame 4 - Cell 4: [ 0  2  3  4  5  6  7  8  9 10 11 12 14 22]
Labels in plot_labels function for frame Frame 6 - Cell 4: [ 0  2  3  4  5  6  7  8  9 10 11 12 14]
Labels in plot_labels function for frame Frame 7 - Cell 4: [ 0  2  3  4  5  6  7  8  9 10 11 12 14]
Labels in plot_labels function for frame Frame 8 - Cell 4: [ 0  2  3  4  5  6  7  8  9 10 11 12]
Labels in plot_labels function for frame Frame 10 - Cell 4: [ 0  2  3  4  5  6  7  8  9 10 11 12]
Labels in plot_labels function for frame Frame 11 - Cell 4: [ 0  2  3  4  5  6  7  8  9 10 11 12 27]
Labels in plot_labels function for frame Frame 19 - Cell 4: [ 0  2  3  4  6  8  9 10 11 39 40 43 44]
Labels in plot_labels function for frame Frame 20 - Cell 4: [ 0  2  3  4  6  8  9 10 11 39 40 43 44]

In [ ]:
import os
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
import imageio
import random

# === PARAMETERS ===
# === PARAMETERS ===
csv_path = "/home/MinaHossain/EmbedTrack/PCA_t-SNE_PHATE/Cells_Centroid_Velocity_MA_5.csv"  # <<< Adjust if needed
image_dir = "/home/MinaHossain/EmbedTrack/Shape_measure/HP4_TRA"  # Directory containing raw mask images
output_dir = "/home/MinaHossain/EmbedTrack/Cell_Frame-Video"

temp_plot_dir = os.path.join(output_dir, "temp_plots")
target_cell_id = 4
output_video = os.path.join(output_dir, f"Cell_{target_cell_id}_Matplotlib_Colored_Video.avi")
output_frame_list = os.path.join(output_dir, f"Cell_{target_cell_id}_CSV_frame_list.txt")
video_fps = 1

# === Ensure directories exist ===
os.makedirs(output_dir, exist_ok=True)
os.makedirs(temp_plot_dir, exist_ok=True)

# === Load CSV data ===
df = pd.read_csv(csv_path)
df_target = df[df["Cell Number"] == target_cell_id]
target_frames = sorted(df_target["Frame"].unique())

# === Create mapping of frame to mask file (use index method) ===
mask_paths = sorted(glob(os.path.join(image_dir, "*.tif")))
mask_map = {}
for idx, path in enumerate(mask_paths):
    frame_num = idx + 1  # Assume index matches frame number
    mask_map[frame_num] = path

# === Assign distinct colors to other cells ===
cell_ids = df["Cell Number"].unique()
cell_colors = {}
for cid in cell_ids:
    if cid == target_cell_id:
        cell_colors[cid] = "red"
    else:
        color = "#" + ''.join([random.choice('0123456789ABCDEF') for _ in range(6)])
        cell_colors[cid] = color

# === Define plot_labels function with distinct colors ===
def plot_labels(original, ax, frame_num, txt_args_base, title=None, cmap='gray'):
    ax.imshow(original, cmap=cmap)
    labels = np.unique(original)
    print(f"Labels in plot_labels function for frame {frame_num}: {labels}")
    ax.set_title(title)
    for label in labels:
        if label != 0:
            inds = np.argwhere(original == label)
            loc = inds.mean(0)
            txt_args = txt_args_base.copy()
            txt_args['color'] = cell_colors.get(label, 'white')
            txt_args['fontsize'] = 'xx-large' if label == target_cell_id else 'large'
            ax.text(loc[1], loc[0], str(label), **txt_args)
    ax.axis("off")

# === Create labeled frames using matplotlib ===
plot_frame_paths = []
for frame_num in target_frames:
    if frame_num not in mask_map:
        print(f"❌ Frame {frame_num} not found in mask map.")
        continue

    mask = imageio.imread(mask_map[frame_num])
    fig, ax = plt.subplots(figsize=(8, 8))
    plot_labels(
        mask, ax, frame_num,
        txt_args_base={'ha': 'center', 'va': 'center', 'fontweight': 'bold'},
        title=f"Frame {frame_num} - Cell {target_cell_id}"
    )
    plot_frame_path = os.path.join(temp_plot_dir, f"frame_{frame_num:03d}.png")
    fig.savefig(plot_frame_path, bbox_inches="tight")
    plt.close(fig)
    plot_frame_paths.append(plot_frame_path)

# === Save final video ===
if plot_frame_paths:
    sample_img = cv2.imread(plot_frame_paths[0])
    height, width, _ = sample_img.shape
    print(f"Video frame size: {width}x{height}")
    fourcc = cv2.VideoWriter_fourcc(*'MJPG')
    out = cv2.VideoWriter(output_video, fourcc, video_fps, (width, height))

    for frame_img_path in plot_frame_paths:
        img = cv2.imread(frame_img_path)
        out.write(img)
    out.release()
    print(f"✅ Final enhanced colorful video saved: {output_video}")
else:
    print("❌ No video frames generated. Video not saved.")

# === Save frame list ===
with open(output_frame_list, "w") as f:
    for frame_num in target_frames:
        f.write(f"{frame_num}\n")

print(f"✅ Frame list saved to: {output_frame_list}")
print(f"✅ Total frames with target cell: {len(target_frames)}")
